In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [50]:
#df = pd.read_csv('train.csv')[['Age','Fare','SibSp','Parch','Survived']]

df=pd.read_csv('titanic_train.csv',usecols=['Age','Fare','SibSp','Parch','Survived']) 

In [51]:
df.head()

,Survived,Age,SibSp,Parch,Fare
0,0,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,1,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,0,35.0,0,0,8.0500


In [52]:
df.dropna(inplace=True)

In [53]:
df.head()

,Survived,Age,SibSp,Parch,Fare
0,0,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,1,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,0,35.0,0,0,8.0500


In [54]:
df['Family']=df['SibSp']+df['Parch']

In [55]:
df.head()

,Survived,Age,SibSp,Parch,Fare,Family
0,0,22.0,1,0,7.2500,1
1,1,38.0,1,0,71.2833,1
2,1,26.0,0,0,7.9250,0
3,1,35.0,1,0,53.1000,1
4,0,35.0,0,0,8.0500,0


In [56]:
df.drop(columns=['SibSp','Parch'], inplace=True)

In [57]:
df.head()

,Survived,Age,Fare,Family
0,0,22.0,7.2500,1
1,1,38.0,71.2833,1
2,1,26.0,7.9250,0
3,1,35.0,53.1000,1
4,0,35.0,8.0500,0


### Train Test Split 

In [58]:
from sklearn.model_selection import train_test_split

In [59]:
X=df.drop(columns='Survived')
X

,Age,Fare,Family
0,22.0,7.2500,1
1,38.0,71.2833,1
2,26.0,7.9250,0
3,35.0,53.1000,1
4,35.0,8.0500,0
...,...,...,...
885,39.0,29.1250,5
886,27.0,13.0000,0
887,19.0,30.0000,0
889,26.0,30.0000,0


In [60]:
y= df['Survived']

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [62]:
X_train.head()

,Age,Fare,Family
328,31.0,20.5250,2
73,26.0,14.4542,1
253,30.0,16.1000,1
719,33.0,7.7750,0
666,25.0,13.0000,0


### <font color='red'> Without binarization </font>

In [63]:
from sklearn.tree import DecisionTreeClassifier

In [64]:
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)


#### Performance Metrics

In [65]:
from sklearn.metrics import accuracy_score

In [66]:
accuracy_score(y_test,y_pred)

0.6223776223776224

### Cross validation

In [67]:
from sklearn.model_selection import cross_val_score

In [68]:
np.mean(cross_val_score(DecisionTreeClassifier(),X,y,cv =10,scoring='accuracy'))

0.647163536776213

### With binarization

 - now lets apply Binarization on 'Family' column where 0 values shows person traveling alone and more than 0 shows it travel with family

In [69]:
from sklearn.preprocessing import Binarizer

In [70]:
from sklearn.compose import ColumnTransformer

In [71]:
trf = ColumnTransformer([
    ('bin',Binarizer(copy=False),['Family'])
    
],remainder='passthrough')

In [72]:
X_train_transformed = trf.fit_transform(X_train)
X_test_transformed =trf.transform(X_test)

In [73]:
X_train_transformed

array([[  1.    ,  31.    ,  20.525 ],
       [  1.    ,  26.    ,  14.4542],
       [  1.    ,  30.    ,  16.1   ],
       ...,
       [  0.    ,  41.    , 134.5   ],
       [  1.    ,  33.    ,  20.525 ],
       [  0.    ,  33.    ,   7.8958]])

In [74]:
pd.DataFrame(X_train_transformed, columns=['Family','Age','Fare'])

,Family,Age,Fare
0,1.0,31.0,20.5250
1,1.0,26.0,14.4542
2,1.0,30.0,16.1000
3,0.0,33.0,7.7750
4,0.0,25.0,13.0000
...,...,...,...
566,1.0,46.0,61.1750
567,0.0,25.0,13.0000
568,0.0,41.0,134.5000
569,1.0,33.0,20.5250


In [75]:
clf = DecisionTreeClassifier()
clf.fit(X_train_transformed,y_train)
y_pred = clf.predict(X_test_transformed)

In [76]:
accuracy_score(y_test,y_pred)

0.6013986013986014

### cross validation

In [77]:
X_trf = trf.fit_transform(X)
np.mean(cross_val_score(DecisionTreeClassifier(),X_trf,y,cv= 10, scoring='accuracy'))

0.6303990610328638